### Description

- this notebook (1) concatenates preprocessed freesurfer data (i.e., resampled to fsaverage and smoothed for each subject), (2) adds annotation network labels (currently using the Yeo 7 Network annotation but can use any annotation file), and (3) filters by network and saves in pickle file
- in a last part of this notebook, also loads the wholebrain fsaverage pickle file and calculates the wholebrain (ie., cortex) measures (surface area and thickness).

In [1]:
#general packages used for data import etc
from nibabel import freesurfer as fs
import nibabel as nib
import pandas as pd
import numpy as np
import os,glob
import re
from joblib import Parallel,delayed
import cPickle as pkl
idx=pd.IndexSlice
datapath=('/Volumes/Users/mbkranz/projects/git_repo_backups/'
          'individual_diff_func_conn_corticalthick/data/')
#subject directory and subject list
SUBJECTS_DIR=('/Volumes/Users/mbkranz/'
              'projects/ACT_Freesurfer_NewProc/')

/Applications/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.chdir(SUBJECTS_DIR)
sublist=!ls -d ACT*_1
#npdata from make_npdata_from_pca.R 
npdata_all=pd.read_csv(datapath+'npdata/np_all.csv',na_values='NA')
#filter out all subs without behav data
sublist_filt=[
    x for x in sublist if int(re.sub("ACT0|ACT|_1","",x)) 
    in npdata_all['subs'].values and x!='ACT6142_1' #bad sub (see notes)
]

In [3]:
meas_key={'thickness':np.int(0),'area':np.int(1)}
hemi_key={'lh':np.int(0),'rh':np.int(1)}
net_key={'network'+str(net):net for net in [1,2,3,4,5,6,7]}
net_key['wholebrain']=[1,2,3,4,5,6,7]

In [4]:
def getmorph(sub,measure,fwhm='10'):
    def get_onehemi(hemi):
        path_data=(SUBJECTS_DIR+ sub+'/surf/'+
                   hemi+'.'+measure+'.'+'fwhm'+fwhm+
                   '.fsaverage.mgh')
        morph_data=(nib.load(path_data)
                    .get_data()
                    .flatten())
        morph_hemi=pd.DataFrame({
            'subs':np.int(re.sub("ACT0|ACT|_1","",sub)),
            'measure':meas_key[measure],
            'hemi':hemi_key[hemi],
            'value':morph_data})
        return morph_hemi
    
    morph_df=pd.concat([get_onehemi(hemi) 
                for hemi in hemi_key.iterkeys()])
    return (morph_df
            .rename_axis('vertex_index')
            .set_index(['hemi'],append=True))

In [13]:
def getannot(annotpath):
    '''loads and concatenates annotation files for 
    both hemispheres and sets vertex and hemi ids
    as indices (keys to join with subject wise metrics)
    '''
    def get_onehemi(hemi):
        annot_data=fs.read_annot(annotpath.format(hemi))
        annot_hemi=pd.DataFrame({
            "annot_label" : annot_data[0],
            "hemi": hemi_key[hemi]})
        return annot_hemi

    annot_df=pd.concat(get_onehemi(hemi) 
                       for hemi in hemi_key.iterkeys())
    return (annot_df
            .rename_axis('vertex_index')
            .set_index(['hemi'],append=True)
            .reorder_levels(['vertex_index','hemi']))

In [14]:
annots=getannot(datapath.replace('data','parcellations')+
                'schaefer2018/FreeSurfer5.3/fsaverage/label/'+
                 '{}.Schaefer2018_400Parcels_17Networks_order.annot')

In [15]:
def average_morph(sub,annots,measure):
    morph=getmorph(sub,measure)
    morph_groupby=(morph.join(annots)
     .set_index(['subs','measure','annot_label'],append=True)
     ['value']
     .groupby(['subs','measure','hemi','annot_label']))
    
    if measure=='thickness':
        morph_summary=morph_groupby.mean()
    elif measure=='area':
        morph_summary=morph_groupby.sum()
    else:
        ValueError('Needs to be either area or thickness')
    return morph_summary

In [16]:
morph_df_allsubs=pd.concat(Parallel(4)(
    delayed(average_morph)
    (sub,annots,meas)
    for sub in sublist_filt
    for meas in meas_key.iterkeys()))

In [39]:
morph_df_allsubs_filt=(
    morph_df_allsubs
    .reset_index()
    .query('annot_label!=0')
    .assign(parc_label=lambda x: (x['annot_label']-1)+(x['hemi']*200),
            measure=lambda x: x['measure'].map({0:'thickness',1:'area'})))

In [46]:
morph_df_allsubs_unstack=(morph_df_allsubs_filt
 .set_index(['subs','measure','hemi','annot_label','parc_label'])
 ['value']
 .unstack('measure')
 .reset_index())

In [47]:
morph_df_allsubs_unstack.to_csv(datapath+'/morphometry/morph_df_400parc_allsubs.csv')